<a href="https://colab.research.google.com/github/Intruder9211/python/blob/main/Uupdatedproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


In [2]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8bhqxc24
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8bhqxc24
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=d2183d9f7b2e274a6146e9783fe08a9ff14e6bb54dc23f16e22682d4385fa3d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-dcddbmki/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

download_with_pydrive = True



In [4]:
pip uninstall pydrive


Found existing installation: PyDrive 1.3.1
Uninstalling PyDrive-1.3.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/PyDrive-1.3.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pydrive/*
Proceed (Y/n)? Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_uninstall.py", line 364, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F

In [5]:
pip install pydrive2


In [6]:
# Old (pydrive)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# New (pydrive2)
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive


In [7]:
import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from google.colab import files
import skimage.io as io
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt


In [8]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [9]:
model_path = "path_to_model.pth"
model = torch.load(model_path, map_location=device)
model.to(device)


<ipython-input-9-448ea01ae353>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_model.pth'

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a simple feedforward neural network
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Fully connected layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply ReLU activation
        x = self.fc2(x)         # Output layer
        return x

# Example instantiation
model = MyModel(input_size=10, hidden_size=20, output_size=2)
print(model)


MyModel(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=2, bias=True)
)


In [11]:
import torchvision.models as models
import torchvision.transforms as T

# Download pretrained Faster R-CNN model (trained on COCO)
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set model to evaluation mode

# Prepare a sample input image
from PIL import Image
image_path = "path_to_image.jpg"
image = Image.open(image_path)
transform = T.Compose([T.ToTensor()])  # Convert image to tensor
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Perform inference
output = model(input_tensor)
print(output)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 77.2MB/s]


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_image.jpg'

In [13]:
import torchvision.models as models
import torchvision.transforms as T

# Download pretrained Faster R-CNN model (trained on COCO)
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set model to evaluation mode

# Prepare a sample input image
from PIL import Image
# Replace "path_to_image.jpg" with the actual path to your image file.
# Make sure the image file exists in the specified location.
# You can upload the file to google colab for testing.
# For example:
# from google.colab import files
# uploaded = files.upload()
# image_path = list(uploaded.keys())[0]
image_path = "/content/deadpool.jpg"
image = Image.open(image_path)
transform = T.Compose([T.ToTensor()])  # Convert image to tensor
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Perform inference
output = model(input_tensor)
print(output)

[{'boxes': tensor([[  94.7661,  292.7259,  490.9296,  698.1817],
        [ 441.0532,    3.5622, 1877.6658, 1160.6997],
        [1168.5099,  219.3003, 1794.0948, 1133.5009],
        [1207.2584,   31.3662, 1854.4673, 1200.0000],
        [1244.4691,  249.9325, 1760.7462, 1113.6947],
        [1626.3865,  144.4955, 1777.0571,  340.0523],
        [1247.7611,  390.4185, 1808.7733, 1169.7520],
        [  98.9822,  563.5470,  273.1262,  689.2894],
        [ 472.2338,  304.2427, 1624.3671,  566.2242],
        [1628.1160,  145.8528, 1777.9849,  353.7515],
        [ 539.4624,  267.0261, 1575.4177, 1134.6953],
        [1563.2897, 1077.7433, 1711.0858, 1197.5128],
        [ 392.4980,  273.8614, 1306.9622, 1155.6715],
        [ 226.0726,  474.5395,  369.2334,  498.0671],
        [   0.0000,  129.3648, 1469.2411, 1166.2354],
        [1531.1431,  592.7039, 1783.8286, 1193.5304],
        [ 481.0584,  125.2542, 1847.6340, 1163.7556],
        [ 278.2023,  313.7299, 1443.5762, 1181.7834],
        [ 176.861

In [15]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Step 1: Load the BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Step 2: Load an image (either from a URL or local file)
image_url = "/content/deadpool.jpg"  # Replace with your image URL
image = Image.open(requests.get(image_url, stream=True).raw)

# Step 3: Process the image and generate a caption
inputs = processor(images=image, return_tensors="pt")
output_ids = model.generate(**inputs)
caption = processor.decode(output_ids[0], skip_special_tokens=True)

# Step 4: Print the generated caption
print("Generated Caption:", caption)


MissingSchema: Invalid URL '/content/deadpool.jpg': No scheme supplied. Perhaps you meant https:///content/deadpool.jpg?

In [21]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Step 1: Load the BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Step 2: Load an image (either from a URL or local file)
image_path = "/content/mike.jpeg"  # Replace with your image URL or local file path
# Use Image.open directly for local files
image = Image.open(image_path)

# Step 3: Process the image and generate a caption
inputs = processor(images=image, return_tensors="pt")
output_ids = model.generate(**inputs)
caption = processor.decode(output_ids[0], skip_special_tokens=True)

# Step 4: Print the generated caption
print("Generated Caption:", caption)

Generated Caption: the fighters are ready to face off against the ufc champion
